In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
CURR_DIR = '/tf/Capstone Project/project-1' #Local Docker image

In [3]:
import pandas as pd
import numpy as np
import unicodedata
import pandas as pd
import numpy as np
import nltk
import unicodedata
import contractions
import os
from datetime import datetime
from num2words import num2words
print(os.getcwd())
os.chdir(CURR_DIR)
print(os.getcwd())

/tf/Capstone Project/Project-1
/tf/Capstone Project/project-1


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
%matplotlib inline
import pingouin as pg
color = sns.color_palette()
from pandas.plotting import table
import scipy
from scipy.stats import pearsonr, spearmanr
from wordcloud import WordCloud, STOPWORDS
import datetime as dt
import plotly
import cufflinks as cf
import plotly.offline
!pwd

/tf/Capstone Project/project-1


In [5]:
def shape_df(df):
    print(f"Number of observations: {df.shape[0]}")
    print(f"Number of variables: {df.shape[1]}")
    print(f"Number of duplicates: {df.duplicated().sum()}")
    print(f"Are there any missing values {df.isnull().values.any()}")
    print("-----")
    print(df.dtypes.sort_values(ascending=True))
    print("------")
    print("Datatypes' proportion:")
    print(df.dtypes.value_counts(ascending=True))

In [6]:
def null_val(df):
    detect_null_val = df.isnull().values.any()
    if detect_null_val:
        null_abs = df.isnull().sum()
        null_pc = df.isnull().sum() / df.isnull().shape[0] *100
        null_concat = pd.concat([null_abs,null_pc], axis=1).round(2)
        null_concat.columns = ['Absolute', 'Percent']
        return null_concat.sort_values(by="Absolute", ascending=False)
    else:
        print("There are no missing values.")

In [7]:
def corrs(x):
    mask = np.triu(x.corr(), 1)
    plt.figure(figsize=(19, 9))
    return sns.heatmap(x.corr(), annot=True, vmax=1, vmin=-1, square=True, cmap='BrBG', mask=mask);

In [8]:
def unique_counts(df, thresh = 15):
    for column in df.columns:
        if df[column].nunique() < thresh:
            print(df.groupby([column], dropna = False).size(), '\n\n')

In [9]:
df_raw = pd.read_csv('./DATA/essay_sample-1.csv')

In [10]:
df_raw.head()

,projectid,teacher_acctid,title,short_description,need_statement,essay
0,fffee68353ade53e7692b23098096683,dddd29e34a0e4d889e65ccaa6efc44cf,Tag Readers for Kindergartners,If our classroom were to get these Tag readers...,"My students need 8 Tag Readers, Tag Reader boo...",My students enjoy hearing stories and this Tag...
1,fffe598a41270538eab48432513513ab,a2abddbac0cc4a11e8d791469c580d2c,Glazed Clay Pots,My students love to open their imaginations by...,My students need 50lbs of clay and some colorf...,My students love to open their imaginations by...
2,fffe426c60834b0ba2eb05f759d854da,0e853a40819781b2de5d410418674d02,Listening For Fluency,Upgrading our listening center by adding indiv...,My students need 6 listening center headphones...,Fifty per cent of my students have been left b...
3,fffe0bb8af3b9cd93046b49653cc923a,820b80005bdffb74fe85e4deb11c5a07,Building Our Library!,Do you know what it is like to have to come to...,My students need 10 subscriptions to Scholasti...,Do you know what it is like to have to come to...
4,fffdf9d286e715165b60674ac9d05c6c,e4787cea9545eb4393e936504e24d91e,Delete the Digital Drama,Did you know that an estimated 32% of teens re...,My students need an Apple iPad to view the Tee...,Did you know that an estimated 32% of teens re...


In [11]:
df_raw.describe()

,projectid,teacher_acctid,title,short_description,need_statement,essay
count,241538,241538,241529,241486,240960,241538
unique,241538,130321,217866,237720,234972,241325
top,e338b56cacda7d972e271421fdf16771,787720cc575e3204991b8ca18d312d2b,Listening Center,Do you remember what it was like the first tim...,My students need a document camera.,Reading is an essential life skill! I am a rea...
freq,1,95,213,48,289,7


In [12]:
shape_df(df_raw)

Number of observations: 241538
Number of variables: 6
Number of duplicates: 0
Are there any missing values True
-----
projectid            object
teacher_acctid       object
title                object
short_description    object
need_statement       object
essay                object
dtype: object
------
Datatypes' proportion:
object    6
dtype: int64


In [13]:
null_val(df_raw)

,Absolute,Percent
need_statement,578,0.24
short_description,52,0.02
title,9,0.00
projectid,0,0.00
teacher_acctid,0,0.00
essay,0,0.00


Considering the total size of the dataset we can drop the data points wiht no need_statement or short_descritpion

In [14]:
#Taking only 50000k datapoints to limit the processing time.
df = df_raw.copy().sort_values('projectid').head(50000) 

### Some text cleaning functions used later

In [15]:
#Removing HTML Tags
from bs4 import BeautifulSoup

def remove_html_tags(text):
    print('Removing HTML Tags, the text can be as big as entire wepage')
    return BeautifulSoup(text, 'html.parser').get_text()


#Removing Accented characters
import unicodedata

def remove_accented_chars(text):
    print("Removing accented characters, which convert résumé to resumve")
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text


# "Well this was fun! See you at 7:30, What do you think!!? #$@@9318@ 🙂🙂🙂" ==> 'Well this was fun See you at  What do you think  '


import re

def remove_special_characters(text, remove_digits=False):
    print("Removing special characters like smileys from the text")
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


#Removing stopwords
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
new_stop_words = []
all_stop_words = stop_words.union(new_stop_words)

not_stopwords = {'no', 'not'}
final_stop_words = set(
    [word for word in all_stop_words if word not in not_stopwords]
)
def remove_stop_words(text):
    new_text = []
    for word in text.split():
        if word not in stop_words:
            new_text.append(word)
    return " ".join(new_text)
            
#Removing symbols (Without apostrophe)
symbols = "!\"#$%&()*+-./:;<=>?,@[\]^_`{|}~\n"
def remove_punctuation(text):
    for i in symbols:
        text = np.char.replace(text, i, ' ')
    return str(text)

#Convers popin' to poping
def convert_to_ing_words(text):
    new_text = []
    for word in text.split():
        if word[-3:] == "in'":
            new_text.append(word[:-3] + "ing")
        else:
            new_text.append(word)
    return " ".join(new_text)

#Conver Numeric values
def convert_numbers_to_words(text):
    new_text = []
    for word in text.split():
        if (word.isnumeric()):
            new_text.extend(num2words(word).split())
        else:
            new_text.append(word)
    return " ".join(new_text)

# My system keeps crashing his crashed yesterday ours crashes daily and presumably we are not lying ==>
#PorterStemmer ==>         my system keep crash hi crash yesterday our crash daili and presum we are not lie
#LancasterStemmer ==>         my system keep crash his crash yesterday our crash dai and presum we ar not lying
ps_ = nltk.porter.PorterStemmer()
ls_ = nltk.stem.LancasterStemmer()
import nltk
def simple_stemming(text, stemmer_type = 'ls'):
    if (stemmer_type == 'ps'):
        stemmer = ps_
    else:
        stemmer = ls_
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    return text
            
    
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def pos_tag_wordnet(tagged_tokens):
    tag_map = {'j': wordnet.ADJ, 'v': wordnet.VERB, 'n': wordnet.NOUN, 'r': wordnet.ADV}
    new_tagged_tokens = [(word, tag_map.get(tag[0].lower(), wordnet.NOUN))
                            for word, tag in tagged_tokens]
    return new_tagged_tokens

def wordnet_lemmatize_text(text):
    wnl = WordNetLemmatizer()
    tagged_tokens = nltk.pos_tag(nltk.word_tokenize(text))
    wordnet_tokens = pos_tag_wordnet(tagged_tokens)
    lemmatized_text = ' '.join(wnl.lemmatize(word, tag) for word, tag in wordnet_tokens)
    return lemmatized_text

#### Checking Lengths

In [16]:
df.columns

Index(['projectid', 'teacher_acctid', 'title', 'short_description',
       'need_statement', 'essay'],
      dtype='object')

In [17]:
def put_word_count_column(df, column_name):
    new_column_name = column_name + '_word_count'
    df[new_column_name] = df[column_name].apply(lambda s: len(str(s).split()))
    return
put_word_count_column(df, 'title')
put_word_count_column(df, 'essay')
put_word_count_column(df, 'short_description')
put_word_count_column(df, 'need_statement')
print(df.shape)
df.describe()

(50000, 10)


,title_word_count,essay_word_count,short_description_word_count,need_statement_word_count
count,50000.000000,50000.00000,50000.000000,50000.000000
mean,5.144860,276.77018,36.990520,19.302440
std,2.241922,88.06406,8.922606,8.903887
min,1.000000,1.00000,1.000000,0.000000
25%,3.000000,211.00000,33.000000,13.000000
50%,5.000000,264.00000,36.000000,18.000000
75%,7.000000,333.00000,41.000000,24.000000
max,21.000000,1210.00000,241.000000,397.000000


#### Text Processing

In [18]:
#existing key-value pairs in contractions library
contractions.slang_dict

{'ima': 'I am going to',
 'gonna': 'going to',
 'gotta': 'got to',
 'wanna': 'want to',
 'woulda': 'would have',
 'gimme': 'give me',
 'asap': 'as soon as possible',
 'u': 'you',
 'r ': 'are ',
 'Im': 'I am',
 "I'm": 'I am',
 'Ima': 'I am about to',
 "Im'a": 'I am about to',
 "I'ma": 'I am about to',
 "I'm'a": 'I am about to',
 'Imo': 'I am going to',
 "Im'o": 'I am going to',
 "I'mo": 'I am going to',
 "I'm'o": 'I am going to',
 'Ive': 'I have',
 "I've": 'I have',
 'Illve': 'I will have',
 "Ill've": 'I will have',
 "I'llve": 'I will have',
 "I'll've": 'I will have',
 'Idve': 'I would have',
 "Id've": 'I would have',
 "I'dve": 'I would have',
 "I'd've": 'I would have',
 'amnt': 'am not',
 "amn't": 'am not',
 'aint': 'are not',
 "ain't": 'are not',
 'arent': 'are not',
 "aren't": 'are not',
 'cause': 'because',
 "'cause": 'because',
 'cant': 'cannot',
 "can't": 'cannot',
 'cantve': 'cannot have',
 "cant've": 'cannot have',
 "can'tve": 'cannot have',
 "can't've": 'cannot have',
 'couldve

In [19]:
def basic_cleaning(input_text):
    text = str(input_text)
    
#   Removing accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
#     Converting to lowercase
    text = text.lower().strip()
    
    #Doing stop words twice once before expanding once, after expanding
    text = remove_stop_words(text)
    
    #Fixing contractions
    text = contractions.fix(text)
    
    text = text.replace("&", 'and')
    text = text.replace("%", " percent")
    
    #Remove punctuation
    text = remove_punctuation(text)
    text = convert_numbers_to_words(text)
    
    #Replacing and character

    
    text = remove_punctuation(text)
    text = convert_to_ing_words(text)
    
    #Second stop words call
    text = remove_stop_words(text)    
    
    text = remove_punctuation(text)
    
    return text

In [20]:
def basic_text_processing(df, column_name):
    print('Processing column: ', column_name)
    new_column_name = column_name + "_changed"
    df[new_column_name] = df[column_name].apply(basic_cleaning)
    return

In [21]:
for col in ['title', 'short_description', 'need_statement', 'essay']: basic_text_processing(df, col)

Processing column:  title
Processing column:  short_description
Processing column:  need_statement
Processing column:  essay


In [22]:
df.columns

Index(['projectid', 'teacher_acctid', 'title', 'short_description',
       'need_statement', 'essay', 'title_word_count', 'essay_word_count',
       'short_description_word_count', 'need_statement_word_count',
       'title_changed', 'short_description_changed', 'need_statement_changed',
       'essay_changed'],
      dtype='object')

In [23]:
df.to_csv('After basic text processing.csv');

In [24]:
df[['need_statement_changed', 'short_description_changed', 'title_changed', 'essay_changed']].head()

,need_statement_changed,short_description_changed,title_changed,essay_changed
241537,students need word building centers twenty sea...,students need incentives make better faster re...,speedy shark reading club,students need incentives make better faster re...
241536,students need high quality books worth chasing...,students need high quality books worth chasing...,go read hrs,students need challenge read high quality exce...
241535,students need ipad mini support instruction st...,kindergarten exciting time learning especially...,technology upgrade must kindergartners ready,kindergarten exciting time learning especially...
241534,students need books native americans including...,remember learning martin luther king jr librar...,innovative lessons civil rights diversity,remember learning martin luther king jr librar...
241533,students need twenty two weather related books...,study weather fifth grade curriculum would lik...,meteorology matters,fabulous children school love research since t...


In [25]:
def stemm_(df, column_name):
    print('Processing column: ', column_name)
    new_column_name = column_name + "_stemmed"
    df[new_column_name] = df[column_name].apply(simple_stemming)
    return

In [26]:
for col in ['need_statement_changed', 'short_description_changed', 'title_changed', 'essay_changed']: stemm_(df, col)

Processing column:  need_statement_changed
Processing column:  short_description_changed
Processing column:  title_changed
Processing column:  essay_changed


In [27]:
df[['need_statement_changed_stemmed','short_description_changed_stemmed', 'title_changed_stemmed', 'essay_changed_stemmed']].head()

,need_statement_changed_stemmed,short_description_changed_stemmed,title_changed_stemmed,essay_changed_stemmed
241537,stud nee word build cent twenty seat stor sack,stud nee int mak bet fast read read system cur...,speedy shark read club,stud nee int mak bet fast read read system cur...
241536,stud nee high qual book wor chas verm goos gir...,stud nee high qual book wor chas verm goos gir...,go read hrs,stud nee challeng read high qual excel book n ...
241535,stud nee ipad min support instruct stay track ...,kindergart excit tim learn espec access child ...,technolog upgrad must kindergartn ready,kindergart excit tim learn espec access child ...
241534,stud nee book nat am includ thirty cop soft ra...,rememb learn martin luth king jr libr classroo...,innov lesson civil right divers,rememb learn martin luth king jr libr classroo...
241533,stud nee twenty two weath rel book act book in...,study weath fif grad curricul would lik stud r...,meteorolog mat,fab childr school lov research sint titl on sc...


In [28]:
df.to_csv('After Stemming.csv')

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizers = {}


In [30]:
def add_vectorize_column(df, column, tv):
    print('Given shape: ',df.shape)
    tv_matrix = tv.fit_transform(df[column].to_numpy())
    tv_matrix = tv_matrix.toarray()
    vocab = [column + '_' + i for i in tv.get_feature_names()]
    vectors = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
    print('Shape of vectors generated: ', vectors.shape)
    return df.join(vectors)

In [31]:
#For Title Column
vectorizers['title'] = TfidfVectorizer(min_df = 0, max_df = 1., use_idf = True, max_features = 200)
df = add_vectorize_column(df, 'title_changed_stemmed', vectorizers['title'])
print('Final shape: ', df.shape)
df.to_csv('After title vectorization.csv')

Given shape:  (50000, 18)
Shape of vectors generated:  (50000, 200)
Final shape:  (50000, 218)


In [32]:
#For need statement
vectorizers['need_statement'] = TfidfVectorizer(min_df = 0, max_df = 1., use_idf = True, max_features = 500)
df = add_vectorize_column(df, 'need_statement_changed_stemmed', vectorizers['need_statement'])
print('Final shape: ', df.shape)
df.to_csv('After need statement vectorization.csv')

Given shape:  (50000, 218)
Shape of vectors generated:  (50000, 500)
Final shape:  (50000, 718)


In [33]:
#For need statement
vectorizers['short_description'] = TfidfVectorizer(min_df = 0, max_df = 1., use_idf = True, max_features = 500)
df = add_vectorize_column(df, 'short_description_changed_stemmed', vectorizers['short_description'])
print('Final shape: ', df.shape)
df.to_csv('After short description vectorisation.csv')

Given shape:  (50000, 718)
Shape of vectors generated:  (50000, 500)
Final shape:  (50000, 1218)


In [ ]:
#For need statement
vectorizers['essays'] = TfidfVectorizer(min_df = 0, max_df = 1., use_idf = True, max_features = 1000)
df = add_vectorize_column(df, 'essay_changed_stemmed', vectorizers['essays'])
print('Final shape: ', df.shape)
df.to_csv('After essays vectorisation.csv')

Given shape:  (50000, 1218)
Shape of vectors generated:  (50000, 1000)
Final shape:  (50000, 2218)
